<h1 style="text-align:center;color:black;">Выбор локации для скважины</h1>

<div class="alert alert-info" role="alert">
Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.
</div>

<h1 style="text-align:center;color:black;">1. Загрузка и подготовка данных</h1>

<div class="alert alert-info" role="alert">
  Импортируем необходимые библиотеки и датасеты:
</div>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
# Создадим функцию для первичного осмотра датасетов
def hello_data(data):
    display(data.head())
    print('----------------------------------------------------')
    print('Информация о данных')
    print(data.info())
    print('----------------------------------------------------')
    print('Числовое описание данных')
    display(data.describe())
    print('----------------------------------------------------')
    print('Коэффициенты корреляции')
    display(data.corr())

In [ ]:
hello_data(data_1)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


----------------------------------------------------
Информация о данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
----------------------------------------------------
Числовое описание данных


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


----------------------------------------------------
Коэффициенты корреляции


,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [ ]:
hello_data(data_2)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


----------------------------------------------------
Информация о данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
----------------------------------------------------
Числовое описание данных


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


----------------------------------------------------
Коэффициенты корреляции


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [ ]:
hello_data(data_3)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


----------------------------------------------------
Информация о данных
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
----------------------------------------------------
Числовое описание данных


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


----------------------------------------------------
Коэффициенты корреляции


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


<h1 style="text-align:center;color:black;">2. Обучение и проверка модели</h1>

In [ ]:
# Создадим функцию моделирования линейной регрессией
def linear_modeling(data):
    features = data.drop(['id', 'product'], axis=1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=27)
    
    display(features_train.shape)
    display(target_train.shape)
    display(features_valid.shape)
    display(target_valid.shape)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse **(0.5)
    mean_product = predicted_valid.mean()
    print(f'rmse = {rmse}')
    print(f'Средний запас предсказанного сырья равен {mean_product}')
    return predicted_valid, target_valid
    

<div class="alert alert-info" role="alert">
  Создадим модель для первого региона
</div>

In [ ]:
predicted_valid_1, target_valid_1 = linear_modeling(data_1)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

rmse = 37.275548178011725
Средний запас предсказанного сырья равен 92.52674412136957


<div class="alert alert-info" role="alert">
  Создадим модель для второго региона
</div>

In [ ]:
predicted_valid_2, target_valid_2 = linear_modeling(data_2)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

rmse = 0.8950796550241396
Средний запас предсказанного сырья равен 68.89267771774594


<div class="alert alert-info" role="alert">
  Создадим модель для третьего региона
</div>

In [ ]:
predicted_valid_3, target_valid_3 = linear_modeling(data_3)

(75000, 3)

(75000,)

(25000, 3)

(25000,)

rmse = 40.05718744735252
Средний запас предсказанного сырья равен 94.97236770725807


<div class="alert alert-info" role="alert">
  Вывод: мы создали модели линейной регрессии для всех трех датасетов. После предсказаний получили следующие результаты:
    <ul>        
         <li>1 регион: rmse = 37.275548178011725; Средний запас предсказанного сырья равен 92.52674412136957</li>
         <li>2 регион: rmse = 0.8950796550241396; Средний запас предсказанного сырья равен 68.89267771774594</li>
         <li>3 регион: rmse = 40.05718744735252; Средний запас предсказанного сырья равен 94.97236770725807</li>
    </ul>
  Таким образом, из результатов модели видно, что больше всего сырья будет в втором и третьем регионе, во втором регионе запас намного меньше.
</div>

<h1 style="text-align:center;color:black;">3. Подготовка к расчёту прибыли</h1>

In [ ]:
COUNT = 200 # количество скважин
BUDGET = 10000000000 # бюджет 
PRICE = 450000 #цена за тысячу баррелей

volume = BUDGET / (PRICE * COUNT)
print(f'Достаточный объем сырья, который должен быть для безубыточного производства = {volume}')

Достаточный объем сырья, который должен быть для безубыточного производства = 111.11111111111111


<div class="alert alert-info" role="alert">
  Вывод: мы определили какого объема достаточно, чтобы добыча была безубыточной. Это как минимум 111 единиц сырья. Наши модели показали, что в среднем в 1 регионе запас сырья 92.53, во 2 регионе 68.90, в 3 регионе 94.97. Таким образом, ни в одном регионе средний запас предсказанного сырья не больше запаса, необходимого для безубыточного производства.

</div>

<h1 style="text-align:center;color:black;">4. Расчёт прибыли и рисков</h1>

In [ ]:
# В функции мы берем 200 наилучших точек и для них рассчитываем прибыль по реальным значениям(target)
def profit(predicted, target):
    budget = 10000000000
    predicted = pd.Series(predicted)
    target = target.reset_index(drop=True)
    top200 = predicted.sort_values(ascending=False).head(200)
    sum_volume = target[top200.index].sum()
    price = 450000
    total_profit = price * sum_volume - budget
    return total_profit

<div class="alert alert-info" role="alert">
    Найдем прибыль для полученного объема сырья в трех регионах:
</div>

In [ ]:
profit_1 = profit(predicted_valid_1, target_valid_1) / 1000000000
print(f'прибыль по 1 региону для предсказанного объема сырья равна {profit_1} млрд')

прибыль по 1 региону для предсказанного объема сырья равна 3.3468645469241296 млрд


In [ ]:
profit_2 = profit(predicted_valid_2, target_valid_2) / 1000000000
print(f'прибыль по 2 региону для предсказанного объема сырья равна {profit_2} млрд')

прибыль по 2 региону для предсказанного объема сырья равна 2.4150866966815108 млрд


In [ ]:
profit_3 = profit(predicted_valid_3, target_valid_3) / 1000000000
print(f'прибыль по 3 региону для предсказанного объема сырья равна {profit_3} млрд')

прибыль по 3 региону для предсказанного объема сырья равна 2.520349163568819 млрд


<div class="alert alert-info" role="alert">
    Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
</div>

In [ ]:
def bootstrap_income(data, product):
    data = pd.Series(data)
    state = np.random.RandomState(27)
    # сохраним значения средней прибыли в переменной values
    values = []
    for i in range(1000):
        subsample = data.sample(n=500, replace=True, random_state=state)
        values.append(profit(subsample, product) / 1000000000)

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    rate = values[values < 0].count() / values.count() *100
    print(f'Средняя прибыль {values.mean():.2f} млрд')
    print(f'Доверительный интервал [{lower:.3f},{upper:.3f}] млрд')
    print(f'Риск убытков {rate:.2f} %')

<div class="alert alert-info" role="alert">
    Используем нашу функцию для расчета по всем регионам:
</div>

In [ ]:
print('Для 1 региона')
bootstrap_income(predicted_valid_1, target_valid_1)

Для 1 региона
Средняя прибыль 0.52 млрд
Доверительный интервал [0.014,1.034] млрд
Риск убытков 2.20 %


In [ ]:
print('Для 2 региона')
bootstrap_income(predicted_valid_2, target_valid_2)

Для 2 региона
Средняя прибыль 0.45 млрд
Доверительный интервал [0.059,0.857] млрд
Риск убытков 1.10 %


In [ ]:
print('Для 3 региона')
bootstrap_income(predicted_valid_3, target_valid_3)

Для 3 региона
Средняя прибыль 0.40 млрд
Доверительный интервал [-0.128,0.904] млрд
Риск убытков 6.90 %


<div class="alert alert-info" role="alert">
    Вывод: Исходя из полученных результатов нам подходят по уровню рисков первый и второй регионы. Но поскольку средняя прибыль в первом регионе больше, то рекомендуем выбрать его для разработки новых скважен.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован